# Azure Read API:
기존 OCR 기능보다 더 정밀하게 텍스트를 블록, 줄(line), 단어(word) 단위로 인식.
Read API는 PDF 파일의 전체 텍스트를 더 잘 정리된 형태로 반환.

pip install azure-cognitiveservices-vision-computervision
pip install msrest
pip install requests


In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time
import os

# Azure Computer Vision 설정
subscription_key = "FQPue8cIR514gwydZdn1xstorLwXFohwwXi0morkCwLxybsl7wnyJQQJ99BAACHYHv6XJ3w3AAAFACOGHyEy"
endpoint = "https://team3-computervision.cognitiveservices.azure.com/"

# Computer Vision Client 생성
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# PDF 파일 경로 설정
pdf_path = "pdf/복강경담낭절제술.pdf"  # OCR을 실행할 입력 파일
input_file_name = os.path.basename(pdf_path).split('.')[0]  # 파일 이름 추출 (확장자 제외)

# 결과 파일 경로 설정
results_folder = "results"  # 결과 파일을 저장할 폴더 이름
os.makedirs(results_folder, exist_ok=True)  # 폴더가 없으면 생성
output_file = os.path.join(results_folder, f"{input_file_name}.txt")  # 저장 파일 경로

# PDF를 OCR 처리 (Read API)
with open(pdf_path, "rb") as pdf_file:
    read_response = computervision_client.read_in_stream(pdf_file, raw=True)

# Operation ID 가져오기
operation_location = read_response.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# OCR 처리 상태 확인
print("OCR 처리 중...")
while True:
    result = computervision_client.get_read_result(operation_id)
    if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
        break
    time.sleep(1)

# 결과 저장
if result.status == OperationStatusCodes.succeeded:
    with open(output_file, "w", encoding="utf-8") as file:
        for page in result.analyze_result.read_results:
            for line in page.lines:
                file.write(line.text + "\n")  # 텍스트만 저장
    print(f"OCR 결과가 '{output_file}' 파일에 저장되었습니다!")
else:
    print("OCR 처리가 실패하였습니다.")

OCR 처리 중...
OCR 결과가 'results\복강경담낭절제술.txt' 파일에 저장되었습니다!


# 확장자에 따라 이미지 파일도 가능하게 변경한 코드

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time
import os

# Azure Computer Vision 설정
subscription_key = "FQPue8cIR514gwydZdn1xstorLwXFohwwXi0morkCwLxybsl7wnyJQQJ99BAACHYHv6XJ3w3AAAFACOGHyEy"
endpoint = "https://team3-computervision.cognitiveservices.azure.com/"

# Computer Vision Client 생성
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# 입력 파일 경로 설정
input_file_path = "img/K-014.jpg"  # OCR을 실행할 입력 파일
input_file_name = os.path.basename(input_file_path).split('.')[0]  # 파일 이름 추출 (확장자 제외)

# 결과 파일 경로 설정
results_folder = "results"  # 결과 파일을 저장할 폴더 이름
os.makedirs(results_folder, exist_ok=True)  # 폴더가 없으면 생성
output_file = os.path.join(results_folder, f"{input_file_name}.txt")  # 저장 파일 경로

# 파일 유형 확인
file_extension = os.path.splitext(input_file_path)[-1].lower()

# OCR 처리 함수 정의
def perform_ocr(file_path, is_pdf):
    with open(file_path, "rb") as file:  # 파일을 바이너리 모드로 열기
        read_response = computervision_client.read_in_stream(file, raw=True)

    # Operation ID 가져오기
    operation_location = read_response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    # OCR 처리 상태 확인
    print("OCR 처리 중...")
    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break
        time.sleep(1)

    return result


# OCR 처리 실행
if file_extension == ".pdf":
    ocr_result = perform_ocr(input_file_path, is_pdf=True)
else:
    ocr_result = perform_ocr(input_file_path, is_pdf=False)

# 결과 저장
if ocr_result.status == OperationStatusCodes.succeeded:
    with open(output_file, "w", encoding="utf-8") as file:
        for page in ocr_result.analyze_result.read_results:
            for line in page.lines:
                file.write(line.text + "\n")  # 텍스트만 저장
    print(f"OCR 결과가 '{output_file}' 파일에 저장되었습니다!")
else:
    print("OCR 처리가 실패하였습니다.")


OCR 처리 중...
OCR 결과가 'results\K-013.txt' 파일에 저장되었습니다!


# 폴더안의 모든 이미지 파일을 변환하는 코드로 변경

In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time
import os

# Azure Computer Vision 설정
subscription_key = "FQPue8cIR514gwydZdn1xstorLwXFohwwXi0morkCwLxybsl7wnyJQQJ99BAACHYHv6XJ3w3AAAFACOGHyEy"
endpoint = "https://team3-computervision.cognitiveservices.azure.com/"

# Computer Vision Client 생성
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# 입력 파일들이 있는 폴더 경로 설정
input_folder_path = "img"  # 이미지 파일들이 저장된 폴더 경로
results_folder = "results"  # 결과 파일을 저장할 폴더 이름
os.makedirs(results_folder, exist_ok=True)  # 결과 폴더 생성

# OCR 처리 함수 정의
def perform_ocr(file_path):
    with open(file_path, "rb") as file:  # 파일을 바이너리 모드로 열기
        read_response = computervision_client.read_in_stream(file, raw=True)

    # Operation ID 가져오기
    operation_location = read_response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    # OCR 처리 상태 확인
    print(f"{file_path} OCR 처리 중...")
    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break
        time.sleep(1)

    return result

# 폴더 내 모든 이미지 파일 처리
supported_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]  # 지원되는 이미지 확장자
image_files = [
    os.path.join(input_folder_path, f)
    for f in os.listdir(input_folder_path)
    if os.path.splitext(f)[-1].lower() in supported_extensions
]

# 이미지 파일별 OCR 실행 및 결과 저장
for image_file in image_files:
    input_file_name = os.path.basename(image_file).split('.')[0]  # 파일 이름 추출 (확장자 제외)
    output_file = os.path.join(results_folder, f"{input_file_name}.txt")  # 저장 파일 경로

    # OCR 수행
    ocr_result = perform_ocr(image_file)

    # 결과 저장
    if ocr_result.status == OperationStatusCodes.succeeded:
        with open(output_file, "w", encoding="utf-8") as file:
            for page in ocr_result.analyze_result.read_results:
                for line in page.lines:
                    file.write(line.text + "\n")  # 텍스트만 저장
        print(f"OCR 결과가 '{output_file}' 파일에 저장되었습니다!")
    else:
        print(f"{image_file} OCR 처리가 실패하였습니다.")


img\K-013.jpg OCR 처리 중...
OCR 결과가 'results\K-013.txt' 파일에 저장되었습니다!
